### 1.Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

In [2]:
data_partitions_dirpath = '../data'
print('Available dataset partitions: ', os.listdir(data_partitions_dirpath))

Available dataset partitions:  ['dev', '.ipynb_checkpoints', 'train', 'test', 'download.sh']


In [3]:
def read_all_shards(partition='dev', data_dir=data_partitions_dirpath):
    shards = []
    for fn in os.listdir(os.path.join(data_dir, partition)):
        with open(os.path.join(data_dir, partition, fn)) as f:
            shards.append(pd.read_csv(f, index_col=None))
    return pd.concat(shards)

test = read_all_shards('test')
dev = read_all_shards('dev')
train = read_all_shards('train')

partitions = {'test': test, 'dev': dev, 'train': train}
for name, df in partitions.items():
    print('Dataset partition "%s" has %d sequences' % (name, len(df)))

Dataset partition "test" has 126171 sequences
Dataset partition "dev" has 126171 sequences
Dataset partition "train" has 1086741 sequences


In [4]:
test.head()

,family_id,sequence_name,family_accession,aligned_sequence,sequence
0,DUF4194,C7MGY1_BRAFD/37-194,PF13835.6,VHLLQGPFLDGRRD...GA.......RYAQLL..RDRTAIEARLAD...,VHLLQGPFLDGRRDGARYAQLLRDRTAIEARLADLFLELIVDDDAQ...
1,Clathrin_propel,Q7SHV2_NEUCR/257-292,PF01394.20,PPEA.SNDFPVALQVSQKYGIIYL.......ITKYGFIHLYDLE,PPEASNDFPVALQVSQKYGIIYLITKYGFIHLYDLE
2,Leu_Phe_trans,K7RWT2_ACIA4/30-205,PF03588.14,...VLAALHEGVFPMPIDGDEVPEPLR.GGMGW.....WSPQL......,VLAALHEGVFPMPIDGDEVPEPLRGGMGWWSPQLRARMPLERIRVP...
3,tRNA_anti-codon,EX7L_BACSU/29-104,PF01336.25,IWIK.GELSNVK...............IHT.RGHIYFT.....LKD...,IWIKGELSNVKIHTRGHIYFTLKDENARMQSVMFARQSERLPFKPE...
4,CSS-motif,PDED_ECOLI/42-242,PF12792.7,NQQRVVQFANHAVE.ELDKVLLPLQA.G...SEVLLP.LIGLPCS....,NQQRVVQFANHAVEELDKVLLPLQAGSEVLLPLIGLPCSVAHLPLR...


In [5]:
train.head()

,family_id,sequence_name,family_accession,aligned_sequence,sequence
0,EKR,R6QAS0_9FIRM/627-685,PF10371.9,.EEKKLVIPTNRPEMKDFVKNILHPIDHLHGDDLPVSKFV..DRAD...,EEKKLVIPTNRPEMKDFVKNILHPIDHLHGDDLPVSKFVDRADGVY...
1,DUF4042,B3MYB3_DROAN/365-542,PF13251.6,KVRISALHLLGSLAKNLPRRFLYGYWHILFPSG.......EHGATN...,KVRISALHLLGSLAKNLPRRFLYGYWHILFPSGEHGATNSHLLLLG...
2,Reg_prop,Q8EJN6_SHEON/296-320,PF07494.11,AQANMETLK..AILF...DKSG.LMWVGGSG,AQANMETLKAILFDKSGLMWVGGSG
3,DUF3880,M1WYU2_PSEP2/257-334,PF12996.7,WFVDNPHLILHHYTHPGTDNTAIFTYDAGNL.EPLRRKGFANTY.Y...,WFVDNPHLILHHYTHPGTDNTAIFTYDAGNLEPLRRKGFANTYYLP...
4,UPRTase,B6GYG1_PENRW/502-699,PF14681.6,AT.DRPAAKLLMTPMRDASI.SGSALRKVHGRVGFYLATELCT.QI...,ATDRPAAKLLMTPMRDASISGSALRKVHGRVGFYLATELCTQIMGL...


#### Prepare for train and test data

In [7]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.utils import to_categorical

In [8]:
vocab = {'.': 0, 'M': 1, 'H': 2, 'B': 3, 'W': 4, 'R': 5, 'U': 6, 
         'I': 7, 'O': 8, 'L': 9, 'T': 10, 'D': 11, 'F': 12, 'X': 13, 
         'Q': 14, 'K': 15, 'N': 16, 'A': 17, 'E': 18, 'Y': 19, 
         'V': 20, 'Z': 21, 'S': 22, 'P': 23, 'C': 24, 'G': 25}

#### Prepare for X

In [ ]:
%%time
X_train_raw, X_test_raw = train['aligned_sequence'].values, test['aligned_sequence'].values
X_train, X_test = [], []

for idx, x in enumerate(X_train_raw):
    if idx % 10000 == 0:
        print("No %d in training part" % idx)
    X_train.append([to_categorical(vocab.get(ch, len(vocab)), len(vocab)) for ch in x])
for idx, x in enumerate(X_test_raw):
    if idx % 10000 == 0:
        print("No %d in testing part" % idx)
    X_test.append([to_categorical(vocab.get(ch, len(vocab)), len(vocab)) for ch in x])
X_train = np.matrix(X_train)
X_test = np.matrix(X_test)

No 0 in training part
No 10000 in training part
No 20000 in training part
No 30000 in training part
No 40000 in training part
No 50000 in training part
No 60000 in training part
No 70000 in training part
No 80000 in training part


In [11]:
from scipy import sparse

In [ ]:
X_train_raw = train['aligned_sequence'].values

X_train = []

max_len = 3000

data = np.ones((3000))
row_index = np.arange((3000))
num = 0
for x in X_train_raw:
    num += 1
    if (num % 100000 == 0):
        print("No %d in training part" % num)
    length = 0
    col_index = []
    for char in x:
        col_index.append(vocab[char])
        length += 1
    if (length > 3000):
        if (length > max_len):
            max_len = length
        continue
    X_train.append(sparse.coo_matrix((data[:length],(row_index[:length], col_index)), shape=(3000, 26)))

In [42]:
%%time
X_train_2 = np.array([x.toarray() for x in X_train])
X_train_2

CPU times: user 2.76 s, sys: 4.61 s, total: 7.38 s
Wall time: 7.38 s


array([[[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [43]:
X_test_raw = test['aligned_sequence'].values

X_test = []

max_len = 3000

data = np.ones((3000))
row_index = np.arange((3000))
num = 0
for x in X_train_raw:
    num += 1
    if (num % 10000 == 0):
        print("No %d in test part" % num)
    length = 0
    col_index = []
    for char in x:
        col_index.append(vocab[char])
        length += 1
    if (length > 3000):
        if (length > max_len):
            max_len = length
        continue
    X_test.append(sparse.coo_matrix((data[:length],(row_index[:length], col_index)), shape=(3000, 26)))

No 10000 in training part
No 20000 in training part
No 30000 in training part
No 40000 in training part
No 50000 in training part
No 60000 in training part
No 70000 in training part
No 80000 in training part
No 90000 in training part
No 100000 in training part
No 110000 in training part
No 120000 in training part
No 130000 in training part
No 140000 in training part
No 150000 in training part
No 160000 in training part
No 170000 in training part
No 180000 in training part
No 190000 in training part
No 200000 in training part
No 210000 in training part
No 220000 in training part
No 230000 in training part
No 240000 in training part
No 250000 in training part
No 260000 in training part
No 270000 in training part
No 280000 in training part
No 290000 in training part
No 300000 in training part
No 310000 in training part
No 320000 in training part
No 330000 in training part
No 340000 in training part
No 350000 in training part
No 360000 in training part
No 370000 in training part
No 380000 

In [44]:
print(max_len)

2866


In [21]:
y_train_raw, y_test_raw = train['family_id'].values, test['family_id'].values

In [22]:
all_fam = {fam: idx for idx, fam in enumerate(list(set(y_train_raw)))}

In [23]:
y_train = np.array([all_fam.get(fam, len(all_fam)) for fam in y_train_raw])
y_test = np.array([all_fam.get(fam, len(all_fam)) for fam in y_test_raw])

In [24]:
y_train

array([ 5683, 17625,  4670, ...,  5133, 16801,  2827])

In [25]:
y_train = np.array([to_categorical(y, num_classes=len(all_fam) for y in y_train)])
y_test = np.array([to_categorical(y, num_classes=len(all_fam) for y in y_test)])

In [25]:
outdir = '../data/preprocess'
np.save(outdir + 'X_train', X_train)
np.save(outdir + 'X_test', X_test)
np.save(outdir + 'y_train', y_train)
np.save(outdir + 'y_test', y_test)